# XGBoost for Cattle Milk Yield Prediction

This notebook implements nested 5-fold cross-validation for XGBoost to predict milk yield.
- Uses nested CV (5-fold outer, 3-fold inner)
- Uses entire dataset (no sampling)
- Expanded hyperparameter grid: n_estimators 200-400, max_depth 3-21 (then more precise during further runs)
- Saves models to models_xgb folder


This code is very similar to modeling.ipynb, but it was specificaly for xgboost model. Steps all the way until model_creation is nearly identical. 

The differences are
- larger parameter grid, now centered around teh optimal parameteres from preliminary testing
- trains and cross validates on entire 210k dataset
- provides inference for test data and produces an output submission.csv

Since XGBoost trains much faster than comparable models, it was easier to have a larger grid and more finer tuned parameters

In [2]:
# Imports
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import KFold, GridSearchCV, ParameterGrid
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
import joblib
from tqdm import tqdm
from itertools import product

# Create models_xgb directory if it doesn't exist
os.makedirs('models_xgb', exist_ok=True)


## Load and Prepare Data


In [3]:
# Load cleaned data
TRAIN_PATH = "cleaned_train_nn.csv"
TEST_PATH = "cleaned_test_nn.csv"

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nTrain columns: {train.columns.tolist()}")


Train shape: (209926, 41)
Test shape: (40000, 41)

Train columns: ['Cattle_ID', 'Age_Months', 'Weight_kg', 'Parity', 'Lactation_Stage', 'Days_in_Milk', 'Feed_Type', 'Feed_Quantity_kg', 'Feeding_Frequency', 'Water_Intake_L', 'Walking_Distance_km', 'Grazing_Duration_hrs', 'Resting_Hours', 'Ambient_Temperature_C', 'Humidity_percent', 'Housing_Score', 'FMD_Vaccine', 'Brucellosis_Vaccine', 'HS_Vaccine', 'BQ_Vaccine', 'Anthrax_Vaccine', 'IBR_Vaccine', 'BVD_Vaccine', 'Rabies_Vaccine', 'Previous_Week_Avg_Yield', 'Body_Condition_Score', 'Milking_Interval_hrs', 'Farm_ID', 'Mastitis', 'Milk_Yield_L', 'Breed_Brown Swiss', 'Breed_Brown Swiss ', 'Breed_Guernsey', 'Breed_Holstein', 'Breed_Holstien', 'Breed_Jersey', 'Management_System_Intensive', 'Management_System_Mixed', 'Management_System_Pastoral', 'Management_System_Semi_Intensive', 'Date_Ordinal']


In [4]:
# Separate features and target
id_cols = ['Cattle_ID'] if 'Cattle_ID' in train.columns else []
target_col = 'Milk_Yield_L'

X_train = train.drop(columns=[target_col] + id_cols, errors='ignore')
y_train = train[target_col]

X_test = test.drop(columns=id_cols, errors='ignore')

print(f"Training features shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test features shape: {X_test.shape}")


Training features shape: (209926, 39)
Training target shape: (209926,)
Test features shape: (40000, 40)


In [5]:
# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"\nNumerical columns ({len(numerical_cols)}): {numerical_cols}")


Categorical columns (1): ['Feed_Type']

Numerical columns (28): ['Age_Months', 'Weight_kg', 'Parity', 'Lactation_Stage', 'Days_in_Milk', 'Feed_Quantity_kg', 'Feeding_Frequency', 'Water_Intake_L', 'Walking_Distance_km', 'Grazing_Duration_hrs', 'Resting_Hours', 'Ambient_Temperature_C', 'Humidity_percent', 'Housing_Score', 'FMD_Vaccine', 'Brucellosis_Vaccine', 'HS_Vaccine', 'BQ_Vaccine', 'Anthrax_Vaccine', 'IBR_Vaccine', 'BVD_Vaccine', 'Rabies_Vaccine', 'Previous_Week_Avg_Yield', 'Body_Condition_Score', 'Milking_Interval_hrs', 'Farm_ID', 'Mastitis', 'Date_Ordinal']


## Preprocessing Pipeline


In [6]:
# Create preprocessing pipeline for tree-based models (no scaling needed)
preprocessor_tree = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_cols)
    ],
    remainder='drop'
)

print("Preprocessing pipeline created")


Preprocessing pipeline created


## Nested Cross-Validation Setup


In [7]:
# Setup nested CV
OUTER_CV = 5
INNER_CV = 3

outer_cv = KFold(n_splits=OUTER_CV, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=INNER_CV, shuffle=True, random_state=42)

print(f"Outer CV folds: {OUTER_CV}")
print(f"Inner CV folds: {INNER_CV}")
print("Using entire dataset (no sampling)")


Outer CV folds: 5
Inner CV folds: 3
Using entire dataset (no sampling)


## Hyperparameter Grid


In [18]:
# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor_tree),
    ('model', XGBRegressor(random_state=42, n_jobs=-1, eval_metric='rmse'))
])

# Optimized hyperparameter grid (reduced size while keeping depth up to 21)
param_grid = {
    'model__n_estimators': [200, 300, 400],
    'model__max_depth': [3, 4, 5],
    'model__learning_rate': [0.1],
    'model__subsample': [0.7, 0.75, 0.8],
    'model__colsample_bytree': [0.7, 0.75, 0.8]
}

# Calculate grid size
grid_size = 1
for values in param_grid.values():
    grid_size *= len(values)
print("Hyperparameter grid:")
for key, values in param_grid.items():
    print(f"  {key}: {values}")
print(f"\nTotal combinations: {grid_size}")
print(f"With {INNER_CV}-fold inner CV: {grid_size * INNER_CV} fits per outer fold")


Hyperparameter grid:
  model__n_estimators: [200, 300, 400]
  model__max_depth: [3, 4, 5]
  model__learning_rate: [0.1]
  model__subsample: [0.7, 0.75, 0.8]
  model__colsample_bytree: [0.7, 0.75, 0.8]

Total combinations: 81
With 3-fold inner CV: 243 fits per outer fold


## Nested Cross-Validation Function


In [19]:
def nested_cv_evaluation(X, y, pipeline, param_grid, outer_cv, inner_cv):
    """
    Perform nested cross-validation for XGBoost.
    Saves each fold's best model to the models_xgb folder.
    """
    print(f"\n{'='*60}")
    print(f"Evaluating XGBoost")
    print(f"{'='*60}")
    print(f"Using entire dataset: {len(X):,} samples")
    
    outer_scores = []
    best_params_list = []
    best_models = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(outer_cv.split(X, y)):
        print(f"\nOuter Fold {fold_idx + 1}/{outer_cv.n_splits}")
        
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Calculate total number of fits for progress display
        param_grid_list = list(ParameterGrid(param_grid))
        n_candidates = len(param_grid_list)
        n_fits = n_candidates * inner_cv.n_splits
        
        # Inner CV for hyperparameter tuning
        print(f"  Testing {n_candidates} parameter combinations ({n_fits} total fits)...")
        
        grid_search = GridSearchCV(
            pipeline,
            param_grid,
            cv=inner_cv,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1  # Use sklearn's built-in progress display
        )
        
        start_time = time.time()
        grid_search.fit(X_train_fold, y_train_fold)
        fit_time = time.time() - start_time
        
        # Evaluate on validation set
        y_pred = grid_search.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val_fold, y_pred)
        r2 = r2_score(y_val_fold, y_pred)
        
        outer_scores.append({
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })
        
        best_params_list.append(grid_search.best_params_)
        best_models.append(grid_search.best_estimator_)
        
        # Save this fold's best model
        model_path = f'models_xgb/xgboost_fold_{fold_idx + 1}.pkl'
        joblib.dump(grid_search.best_estimator_, model_path)
        print(f"  Saved model to {model_path}")
        
        print(f"  Best params: {grid_search.best_params_}")
        print(f"  Validation RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
        print(f"  Fit time: {fit_time:.2f}s ({fit_time/60:.2f} minutes)")
    
    # Aggregate results
    avg_rmse = np.mean([s['rmse'] for s in outer_scores])
    avg_mae = np.mean([s['mae'] for s in outer_scores])
    avg_r2 = np.mean([s['r2'] for s in outer_scores])
    std_rmse = np.std([s['rmse'] for s in outer_scores])
    
    print(f"\nXGBoost Results:")
    print(f"  Average RMSE: {avg_rmse:.4f} (±{std_rmse:.4f})")
    print(f"  Average MAE: {avg_mae:.4f}")
    print(f"  Average R²: {avg_r2:.4f}")
    
    return {
        'outer_scores': outer_scores,
        'best_params': best_params_list,
        'best_models': best_models,
        'avg_rmse': avg_rmse,
        'avg_mae': avg_mae,
        'avg_r2': avg_r2,
        'std_rmse': std_rmse
    }


## Run Nested Cross-Validation


In [21]:
# Run nested CV
start_time = time.time()

results = nested_cv_evaluation(
    X_train, y_train,
    pipeline,
    param_grid,
    outer_cv,
    inner_cv
)

total_time = time.time() - start_time
print(f"\nTotal time: {total_time/60:.2f} minutes ({total_time/3600:.2f} hours)")



Evaluating XGBoost
Using entire dataset: 209,926 samples

Outer Fold 1/5
  Testing 81 parameter combinations (243 total fits)...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
  Saved model to models_xgb/xgboost_fold_1.pkl
  Best params: {'model__colsample_bytree': 0.75, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 300, 'model__subsample': 0.7}
  Validation RMSE: 4.1200, MAE: 3.1991, R²: 0.4088
  Fit time: 61.94s (1.03 minutes)

Outer Fold 2/5
  Testing 81 parameter combinations (243 total fits)...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
  Saved model to models_xgb/xgboost_fold_2.pkl
  Best params: {'model__colsample_bytree': 0.7, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 300, 'model__subsample': 0.75}
  Validation RMSE: 4.1086, MAE: 3.2044, R²: 0.4084
  Fit time: 60.58s (1.01 minutes)

Outer Fold 3/5
  Testing 81 parameter combinations (243 total fits)...
Fitting 3 folds for each of 81

## Results Summary


In [22]:
# Display results summary
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)
print(f"Average RMSE: {results['avg_rmse']:.4f} (±{results['std_rmse']:.4f})")
print(f"Average MAE: {results['avg_mae']:.4f}")
print(f"Average R²: {results['avg_r2']:.4f}")

print("\nBest parameters for each fold:")
for i, params in enumerate(results['best_params']):
    print(f"\nFold {i+1}:")
    for key, value in params.items():
        print(f"  {key}: {value}")

# Get most common best parameters across folds
def get_most_common_params(best_params_list):
    """Get the most common parameter values across folds"""
    if not best_params_list:
        return {}
    
    param_counts = {}
    for params in best_params_list:
        for key, value in params.items():
            if key not in param_counts:
                param_counts[key] = {}
            if value not in param_counts[key]:
                param_counts[key][value] = 0
            param_counts[key][value] += 1
    
    most_common = {}
    for key, value_counts in param_counts.items():
        most_common[key] = max(value_counts, key=value_counts.get)
    
    return most_common

best_params_final = get_most_common_params(results['best_params'])
print("\n" + "="*60)
print("MOST COMMON BEST PARAMETERS (across all folds):")
print("="*60)
for key, value in best_params_final.items():
    print(f"  {key}: {value}")



RESULTS SUMMARY
Average RMSE: 4.1164 (±0.0073)
Average MAE: 3.2014
Average R²: 0.4065

Best parameters for each fold:

Fold 1:
  model__colsample_bytree: 0.75
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__subsample: 0.7

Fold 2:
  model__colsample_bytree: 0.7
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__subsample: 0.75

Fold 3:
  model__colsample_bytree: 0.8
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__subsample: 0.7

Fold 4:
  model__colsample_bytree: 0.75
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__subsample: 0.75

Fold 5:
  model__colsample_bytree: 0.75
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__subsample: 0.75

MOST COMMON BEST PARAMETERS (across all folds):
  model__colsample_bytree: 0.75
  model__learning_rate: 0.1
  model__max_depth: 3
  model__n_estimators: 300
  model__sub

## Retrain on Entire Dataset with Best Parameters

We then pick the best parameters to be the most common parameters (mode) out of all 5 folds, and then we retrain a model on the entire dataset (no train/test split).


In [23]:
# Create final pipeline with best parameters
final_pipeline = Pipeline([
    ('preprocessor', preprocessor_tree),
    ('model', XGBRegressor(random_state=42, n_jobs=-1, eval_metric='rmse'))
])

# Set best parameters
final_pipeline.set_params(**best_params_final)

print("Retraining on entire dataset with best hyperparameters...")
print(f"Training samples: {len(X_train):,}")

start_time = time.time()
final_pipeline.fit(X_train, y_train)
fit_time = time.time() - start_time

print(f"Training completed in {fit_time:.2f} seconds ({fit_time/60:.2f} minutes)")

# Evaluate on training data
y_train_pred = final_pipeline.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("\nFinal Model Training Performance:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE: {train_mae:.4f}")
print(f"  R²: {train_r2:.4f}")

# Save final model
final_model_path = 'models_xgb/xgboost_final.pkl'
joblib.dump(final_pipeline, final_model_path)
print(f"\nFinal model saved to {final_model_path}")


Retraining on entire dataset with best hyperparameters...
Training samples: 209,926
Training completed in 1.28 seconds (0.02 minutes)

Final Model Training Performance:
  RMSE: 4.0825
  MAE: 3.1756
  R²: 0.4163

Final model saved to models_xgb/xgboost_final.pkl


## Generate Predictions on Test Data


We then produce the output to the submission_xgboost.csv

In [24]:
# Generate predictions on test data using final model
print("Generating predictions on test data...")
test_predictions = final_pipeline.predict(X_test)

print(f"Predictions shape: {test_predictions.shape}")
print(f"Predictions range: [{test_predictions.min():.2f}, {test_predictions.max():.2f}]")
print(f"Predictions mean: {test_predictions.mean():.2f}")
print(f"Predictions std: {test_predictions.std():.2f}")

# Create submission file
if 'Cattle_ID' in test.columns:
    submission = pd.DataFrame({
        'Cattle_ID': test['Cattle_ID'],
        'Milk_Yield_L': test_predictions
    })
else:
    # If no Cattle_ID, create sequential IDs
    submission = pd.DataFrame({
        'Cattle_ID': range(1, len(test_predictions) + 1),
        'Milk_Yield_L': test_predictions
    })

# Save submission
submission_path = 'submission_xgboost.csv'
submission.to_csv(submission_path, index=False)
print(f"\nSubmission file saved to: {submission_path}")
print(f"Submission shape: {submission.shape}")
print("\nFirst few predictions:")
print(submission.head(10))


Generating predictions on test data...
Predictions shape: (40000,)
Predictions range: [4.38, 27.67]
Predictions mean: 15.60
Predictions std: 3.36

Submission file saved to: submission_xgboost.csv
Submission shape: (40000, 2)

First few predictions:
   Cattle_ID  Milk_Yield_L
0          1     18.817492
1          2     10.555753
2          3     22.754541
3          4     15.351189
4          5     18.558693
5          6     19.449032
6          7     15.052192
7          8     18.379040
8          9     21.967062
9         10     14.004067
